In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.dqn import DQN
from networks.lstm_dueling import LSTMDueling

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch

In [2]:
norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 20
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/vinay/code/git/smart-broker/notebooks/../environment.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['rolling_price'] = self.df[self.price_typ].rolling(self.roll_period).sum()
/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [ ]:
batch_size = 10
target_net = LSTMDueling(
    input_dim=batch_dur*3+3,
    output_dim=3,
)

policy_net = LSTMDueling(
    input_dim=batch_dur*3+3,
    output_dim=3,
)

dqn = DQN(
    env=env,
    env_type='vector',
    n_actions=3,
    log_freq=1,
    train_freq=3,
    batch_size=batch_size,
    w_sync_freq=1,
    memory_size=500,
    gamma=0.9995,
    step_size=0.01,
    episodes=500,
    target_net=target_net,
    policy_net=policy_net,
    loss_func=nn.SmoothL1Loss(),
    optimizer=torch.optim.Adam(policy_net.parameters(), lr=0.00008),
    load_pretrained=False,
    save_pretrained=False,
    model_path='../models/dqn_d_lstm',
    network_type='lstm',
)

dqn.run(500)

collecting experience...

Ep: 0 | TS: 19970 | L: 13978.658 | R: -4.33 | P: 19.39 | R.Avg P: 19.39 | NW: 119.39 | R.Avg NW: 119.39 | R.U: 217
Ep: 1 | TS: 39940 | L: 11615.479 | R: -4.3 | P: 1.54 | R.Avg P: 10.46 | NW: 101.54 | R.Avg NW: 110.46 | R.U: 203
Ep: 2 | TS: 59910 | L: 11367.894 | R: -4.3 | P: -6.51 | R.Avg P: 4.81 | NW: 93.49 | R.Avg NW: 104.81 | R.U: 193
Ep: 3 | TS: 79880 | L: 11427.285 | R: -4.26 | P: 10.81 | R.Avg P: 6.31 | NW: 110.81 | R.Avg NW: 106.31 | R.U: 196
Ep: 4 | TS: 99850 | L: 11541.326 | R: -4.17 | P: 32.29 | R.Avg P: 11.5 | NW: 132.29 | R.Avg NW: 111.5 | R.U: 205
Ep: 5 | TS: 119820 | L: 11181.718 | R: -4.36 | P: -8.33 | R.Avg P: 8.2 | NW: 91.67 | R.Avg NW: 108.2 | R.U: 200
Ep: 6 | TS: 139790 | L: 12306.84 | R: -4.23 | P: 26.6 | R.Avg P: 10.83 | NW: 126.6 | R.Avg NW: 110.83 | R.U: 204
Ep: 7 | TS: 159760 | L: 12044.143 | R: -4.28 | P: 13.42 | R.Avg P: 11.15 | NW: 113.42 | R.Avg NW: 111.15 | R.U: 204
Ep: 8 | TS: 179730 | L: 11990.035 | R: -4.2 | P: 34.16 | R.Avg P: 

In [ ]:
s = env.reset()
s

In [ ]:
action = dqn._get_action(s, 0)
action

In [ ]:
s, _, _, _ = env.step(1)
s

### Visualizations

### Traning

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in dqn.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

### Evaluation

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in dqn.eval_logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(dqn.target_net.state_dict(), '../models/dqn_d_lstm/target_net')
# torch.save(dqn.policy_net.state_dict(), '../models/dqn_d_lstm/policy_net')